In [1]:
import wandb

In [2]:
wandb.login()

wandb: Currently logged in as: zkdeng (zkdeng-university-of-arizona) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [3]:
import os
import sys
from pathlib import Path

import json
import datetime
import albumentations as A
import numpy as np
import torch
import seaborn as sns
import matplotlib.pyplot as plt
import wandb
import warnings
import argparse
import time
import random

from PIL import Image
from albumentations.pytorch import ToTensorV2
from anomalib.data import MVTecAD
from anomalib.deploy import ExportType
from anomalib.engine import Engine
from anomalib.models import Patchcore
from anomalib.data import Folder
from anomalib.loggers import AnomalibWandbLogger

from pytorch_lightning import seed_everything, Trainer
# 2. Seed python random, numpy, torch (CPU & GPU)
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

# 3. cuDNN settings (for deterministic behavior)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# 4. If using PyTorch >= something, enforce deterministic algorithms
# (this may make some ops complain if no deterministic variant exists)
try:
    torch.use_deterministic_algorithms(True)
except AttributeError:
    pass

# 5. In PyTorch Lightning, use seed_everything, with worker seeding
seed_everything(seed, workers=True)

def parse_layers(s: str):
    s = s.strip()
    return json.loads(s) if s.startswith("[") else [x.strip() for x in s.split(",") if x.strip()]

def albu_adapter(aug):
    """Wrap an Albumentations Compose so it accepts (image) and returns a Tensor.
       Converts PIL/Tensor to NumPy HxWxC uint8 before calling Albumentations.
    """
    def _call(image):
        # 1) Normalize input type to NumPy HxWxC
        if isinstance(image, np.ndarray):
            img = image
            # If CHW, convert to HWC
            if img.ndim == 3 and img.shape[0] in (1, 3, 4) and img.shape[2] not in (1, 3, 4):
                img = np.transpose(img, (1, 2, 0))
        elif isinstance(image, Image.Image):
            img = np.array(image)  # PIL -> HWC uint8
        elif isinstance(image, torch.Tensor):
            arr = image.detach().cpu().numpy()
            # Assume CHW; convert to HWC
            if arr.ndim == 3 and arr.shape[0] in (1, 3, 4):
                arr = np.transpose(arr, (1, 2, 0))
            # If float in [0,1], convert to uint8 0..255 for Albumentations Normalize defaults
            if arr.dtype != np.uint8:
                arr = np.clip(arr, 0, 1) * 255.0
                arr = arr.astype(np.uint8)
            img = arr
        else:
            raise TypeError(f"Unsupported image type: {type(image)}. Expected numpy, PIL, or torch.Tensor.")

        if img.ndim == 2:
            img = img[..., None]

        # 2) Call Albumentations with named argument
        out = aug(image=img)
        
        return out["image"]
    return _call

def define_transforms(image_size):
    train_aug = A.Compose([
        A.Resize(image_size, image_size),
        # Optional robustifying augs for field images:
        # A.HorizontalFlip(p=0.5),
        # A.ColorJitter(p=0.2),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),  # expects uint8 input
        ToTensorV2(),
    ])

    eval_aug = A.Compose([
        A.Resize(image_size, image_size),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2(),
    ])

    train_tf = albu_adapter(train_aug)
    eval_tf  = albu_adapter(eval_aug)

    return train_tf, eval_tf

def wandb_init(args):
    wandb_logger = AnomalibWandbLogger(
        project="spider-anomaly-detection",
        name="patchcore-spider-experiment",
        save_dir="./logs"
    )

    wandb.init(
        project=os.getenv("WANDB_PROJECT", "spider-novelty"),
        name="patchcore-spider-experiment",
        tags=["patchcore", "spiders", "anomaly-detection", "resnet18"],
        notes="Spider anomaly detection using PatchCore with detailed evaluation",
        config={
            "model": "PatchCore",
            "backbone": args.backbone,
            "layers": args.layers,
            "coreset_sampling_ratio": args.coreset_sampling_ratio,
            "num_neighbors": args.num_neighbors,
            "train_batch_size": args.train_batch_size,
            "eval_batch_size": args.eval_batch_size,
            "max_epochs": 1,
            "dataset_root": args.dataset_root,
            "image_size": args.image_size,
            "num_workers": args.num_workers,
        }
    )

    print("Weights & Biases logger initialized!")
    print(f"Project: {wandb.run.project}")
    print(f"Run name: {wandb.run.name}")
    print(f"Run URL: {wandb.run.url}")
    return wandb_logger

def post_process_results(post_processor):
    threshold_value = None
    threshold_attrs = ['threshold', 'threshold_', 'image_threshold', 'pixel_threshold']
    for attr in threshold_attrs:
        if hasattr(post_processor, attr):
            threshold_value = getattr(post_processor, attr)
            break
    
    normalized_threshold_value = None
    normalized_threshold_attrs = ['normalized_image_threshold']
    for attr in normalized_threshold_attrs:
        if hasattr(post_processor, attr):
            normalized_threshold_value = getattr(post_processor, attr)
            break

    return threshold_value, normalized_threshold_value

def main(config=None):
    # Define default configuration
    default_config = {
        'num_neighbors': 9,
        'backbone': 'resnet18',
        'layers': ['layer3', 'layer4'],
        'coreset_sampling_ratio': 0.05,
        'train_batch_size': 16,
        'eval_batch_size': 16,
        'num_workers': 0,
        'training_dir': '/Users/zideng/Work/Anomaly/data/black_widows_200',
        'model': 'PatchCore',
        'image_size': 256,
        'max_epochs': 1
    }
    
    # If config is provided, update defaults
    if config is not None:
        default_config.update(config)
    
    # Use the merged config
    args = default_config
    
    start_time = time.time()
    
    train_tf, eval_tf = define_transforms(args['image_size'])

    with wandb.init(config=args):
        # Get the config from wandb (this will have sweep parameters if running in a sweep)
        config = wandb.config
        
        # Create wandb logger
        wandb_logger = AnomalibWandbLogger(
            project="spider-anomaly-detection",
            name="patchcore-spider-experiment", 
            save_dir="./logs"
        )

        dm = Folder(
            name="spider_anomaly_detection",
            root=config.training_dir,
            normal_dir="normal",
            abnormal_dir="abnormal",
            train_augmentations=train_tf,
            val_augmentations=eval_tf,
            test_augmentations=eval_tf,
            train_batch_size=config.train_batch_size,
            eval_batch_size=config.eval_batch_size,
            num_workers=config.num_workers,
        )

        model = Patchcore(
            backbone=config.backbone,
            layers=config.layers,
            pre_trained=True,
            coreset_sampling_ratio=config.coreset_sampling_ratio,
            num_neighbors=config.num_neighbors,
            visualizer=False,
        )

        engine = Engine(
            max_epochs=config.max_epochs,
            accelerator="auto",
            devices=1,
            logger=wandb_logger,
            enable_progress_bar=True,
            log_every_n_steps=1,
            enable_checkpointing=True,
        )

        print("🚀 Starting training with wandb logging...")
        engine.fit(datamodule=dm, model=model)

        print("🧪 Running evaluation with wandb logging...")
        test_results = engine.test(datamodule=dm, model=model)

        threshold_value, normalized_threshold_value = post_process_results(model.post_processor)

        basic_metrics = {
            "test/image_AUROC": test_results[0].get("image_AUROC", 0),
            "test/image_F1Score": test_results[0].get("image_F1Score", 0),
            "test/threshold_value": threshold_value,
            "test/threshold_type": type(model.post_processor).__name__,
            "test/model_configuration": dict(config),
            "training/duration_seconds": time.time() - start_time,
        }

        wandb.log(basic_metrics)


/Users/zideng/mamba/envs/ml/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/zideng/mamba/envs/ml/lib/python3.13/site-packages/openvino/runtime/__init__.py:10: DeprecationWarning: The `openvino.runtime` module is deprecated and will be removed in the 2026.0 release. Please replace `openvino.runtime` with `openvino`.
  warnings.warn(
Seed set to 42


In [ ]:
sweep_config = {
    'method': 'bayesian',
    'metric': {
        'name': 'image_AUROC',
        'goal': 'maximize'
    },
    'parameters': {
        'num_neighbors': {
            'values': [15]
        },
        'backbone': {
            'values': ['resnet18']
        },
        'layers': {
            'values': [['layer1', 'layer4']]
        },
        'coreset_sampling_ratio': {
            'values': [0.05, 0.1, 0.15, 0.2, 0.25, 0.3]
        },
        'train_batch_size': {
            'values': [16]
        },  
        'eval_batch_size': {
            'values': [16]
        },
        'num_workers': {
            'values': [0]
        },
        'training_dir': {
            'values': ['/Users/zideng/Work/Anomaly/data/black_widows_200']
        },
        'model': {
            'values': ['PatchCore']
        },
        'image_size': {
            'values': [256]
        },
        'max_epochs': {
            'values': [1]
        }
    }
}
sweep_id = wandb.sweep(sweep_config, project="spider-novelty")

Create sweep with ID: me7gffum
Sweep URL: https://wandb.ai/zkdeng-university-of-arizona/spider-novelty/sweeps/me7gffum


In [5]:
wandb.agent(sweep_id, main, count=4)

wandb: Agent Starting Run: 75nldr84 with config:
wandb: 	backbone: resnet18
wandb: 	coreset_sampling_ratio: 0.15
wandb: 	eval_batch_size: 16
wandb: 	image_size: 256
wandb: 	layers: ['layer1', 'layer4']
wandb: 	max_epochs: 1
wandb: 	model: PatchCore
wandb: 	num_neighbors: 15
wandb: 	num_workers: 0
wandb: 	train_batch_size: 16
wandb: 	training_dir: /Users/zideng/Work/Anomaly/data/black_widows_200


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/zideng/mamba/envs/ml/lib/python3.13/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/Users/zideng/mamba/envs/ml/lib/python3.13/site-packages/lightning/pytorch/core/optimizer.py:183: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer

  | Name           | Type           | Params | Mode 
----------------------------------------------------------
0 | pre_processor  | PreProcessor   | 0      | train
1 | post_processor | PostProcessor  | 0      | train
2 | evaluator      | Evaluator      | 0      | train
3 | model          | PatchcoreModel | 11.2 M | train
----------------------------------------------------------
11.2 M    Trainable param

🚀 Starting training with wandb logging...


/Users/zideng/mamba/envs/ml/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.
/Users/zideng/mamba/envs/ml/lib/python3.13/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Epoch 0: 100%|██████████| 10/10 [00:01<00:00,  7.48it/s, v_num=dr84]

...

wandb: ERROR Run 8f1zchbl errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/Users/zideng/mamba/envs/ml/lib/python3.13/site-packages/wandb/agents/pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR     ~~~~~~~~~~~~~~^^
wandb: ERROR   File "/var/folders/pv/r44bwgrn3sz8b3s7crkjg8800000gn/T/ipykernel_35968/1819891171.py", line 232, in main
wandb: ERROR     engine.fit(datamodule=dm, model=model)
wandb: ERROR     ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^
wandb: ERROR   File "/Users/zideng/mamba/envs/ml/lib/python3.13/site-packages/anomalib/engine/engine.py", line 416, in fit
wandb: ERROR     self.trainer.fit(model, train_dataloaders, val_dataloaders, datamodule, ckpt_path)
wandb: ERROR     ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
wandb: ERROR   File "/Users/zideng/mamba/envs/ml/lib/python3.13/site-packages/lightning/pytorch/trainer/trainer.py", line 561, in fit
wandb: ERROR     call._call_and_ha

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/zideng/mamba/envs/ml/lib/python3.13/site-packages/lightning/pytorch/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/Users/zideng/mamba/envs/ml/lib/python3.13/site-packages/lightning/pytorch/core/optimizer.py:183: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer

  | Name           | Type           | Params | Mode 
----------------------------------------------------------
0 | pre_processor  | PreProcessor   | 0      | train
1 | post_processor | PostProcessor  | 0      | train
2 | evaluator      | Evaluator      | 0      | train
3 | model          | PatchcoreModel | 11.2 M | train
----------------------------------------------------------
11.2 M    Trainable param

🚀 Starting training with wandb logging...
Epoch 0: 100%|██████████| 10/10 [00:01<00:00,  5.70it/s, v_num=fzcd]

wandb-core(79079) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
Traceback (most recent call last):
  File "/var/folders/pv/r44bwgrn3sz8b3s7crkjg8800000gn/T/ipykernel_35968/1819891171.py", line 232, in main
    engine.fit(datamodule=dm, model=model)
    ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/zideng/mamba/envs/ml/lib/python3.13/site-packages/anomalib/engine/engine.py", line 416, in fit
    self.trainer.fit(model, train_dataloaders, val_dataloaders, datamodule, ckpt_path)
    ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/zideng/mamba/envs/ml/lib/python3.13/site-packages/lightning/pytorch/trainer/trainer.py", line 561, in fit
    call._call_and_handle_interrupt(
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^
        self, self._fit_impl, model, train_dataloaders, val_dataloaders, datamodule, ckpt_path
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 

wandb: ERROR Run 5ho7fzcd errored:
wandb: ERROR Traceback (most recent call last):
wandb: ERROR   File "/Users/zideng/mamba/envs/ml/lib/python3.13/site-packages/wandb/agents/pyagent.py", line 306, in _run_job
wandb: ERROR     self._function()
wandb: ERROR     ~~~~~~~~~~~~~~^^
wandb: ERROR   File "/var/folders/pv/r44bwgrn3sz8b3s7crkjg8800000gn/T/ipykernel_35968/1819891171.py", line 232, in main
wandb: ERROR     engine.fit(datamodule=dm, model=model)
wandb: ERROR     ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^
wandb: ERROR   File "/Users/zideng/mamba/envs/ml/lib/python3.13/site-packages/anomalib/engine/engine.py", line 416, in fit
wandb: ERROR     self.trainer.fit(model, train_dataloaders, val_dataloaders, datamodule, ckpt_path)
wandb: ERROR     ~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
wandb: ERROR   File "/Users/zideng/mamba/envs/ml/lib/python3.13/site-packages/lightning/pytorch/trainer/trainer.py", line 561, in fit
wandb: ERROR     call._call_and_ha